In [22]:
#pip install googledrivedownloader
#!pip install PyMuPDF
#!pip install google-auth google-auth-oauthlib google-auth-httplib2
#!pip install google-api-python-client
#!pip install biopython


In [10]:
from google_drive_downloader import GoogleDriveDownloader as gdd
import os
import PyPDF2
import fitz
from __future__ import print_function
import os.path
import io
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.http import MediaFileUpload
from Bio import Entrez,  Medline


In [3]:
os.getcwd()

'/Users/hammedadedamolaakande/Library/CloudStorage/OneDrive-ConcordiaUniversity-Canada/Data_Science_projects/Brave_Career/final_project/Chatbot_AI/Scripts'

In [11]:
#set working directory as "final_project"

wd = '/Users/hammedadedamolaakande/Library/CloudStorage/OneDrive-ConcordiaUniversity-Canada/Data_Science_projects/Brave_Career/final_project'

os.chdir(wd)

In [16]:
#This part is to download ALL the PDFs from the drive to local machine
#Nothing to modify here except the folder ID (if its different from our current project, other remains the same)
#Also, token should be deleted especially if you modify the SCOPES


# If modifying these SCOPES, delete the file token.json.
#SCOPES = ['https://www.googleapis.com/auth/drive']
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

def create_service():
    creds = None
    # token.json stores my access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, I'm gonna log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('drive', 'v3', credentials=creds)
    return service

# save service instance to interact with Google Drive
service = create_service()



def download_all_pdfs(service, folder_id):
    page_token = None
    while True:
        # Query to list the PDF files in our Google Drive folder
        results = service.files().list(
            q=f"'{folder_id}' in parents and mimeType='application/pdf'",
            pageSize=10,  # Keep as 10 (for fast download)
            fields="nextPageToken, files(id, name)",
            pageToken=page_token  # Use the nextPageToken from the previous request
        ).execute()

        items = results.get('files', [])
        if not items:
            print('No files found.')
            break  # Exit if no files are found

        for item in items:
            file_name = item['name']
            # Check if the file already exists in the current directory
            if not os.path.exists(file_name):
                print(f"Downloading {file_name}...")
                request = service.files().get_media(fileId=item['id'])
                fh = io.BytesIO()
                downloader = MediaIoBaseDownload(fh, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()
                    print(f"Download {int(status.progress() * 100)}%.")

                # Save the file to disk only if it doesn't already exist
                
                with open(f'PDF_docs/{file_name}', 'wb') as f:
                    fh.seek(0)
                    f.write(fh.read())
            else:
                print(f"{file_name} already exists. Skipping download.")

        page_token = results.get('nextPageToken', None)
        if page_token is None:
            break 

            
download_all_pdfs(service, '1n_9nLO15KnypYNEHbj9Z8USrBoQZVliQ')  

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=771976957298-11sjv4tsimjnl9sau0mb37sjapveeo54.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52701%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.readonly&state=bRGIHz2MNUHWzOjCqpjcwRrh0U0Ntm&access_type=offline
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.


In [20]:
#extract data from the current wd- convert the PDFs to text


def is_pdf_file(filename):
    return filename.lower().endswith('.pdf')

def convert_pdf_to_text(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text()
    return text

def process_pdfs_in_directory(directory_path):
    for filename in os.listdir(directory_path):
        if is_pdf_file(filename):
            full_path = os.path.join(directory_path, filename)
            pdf_text = convert_pdf_to_text(full_path)
            
            # Define a new file name for the output with .txt extension
            output_filename = filename.rsplit('.', 1)[0] + '.txt'
            output_path = os.path.join(directory_path, output_filename)
            
            # Write the extracted text to a new text file
            with open(output_path, 'w', encoding='utf-8') as text_file:
                text_file.write(pdf_text)
            
            print(f"Processed and saved: {filename} as {output_filename}")
            
# Set directory_path to the folder containing the PDFs
directory_path = os.path.join(os.getcwd(), 'PDF_docs')
process_pdfs_in_directory(directory_path)



Processed and saved: Diabetic_Diet.pdf as Diabetic_Diet.txt
Processed and saved: Changing_Diabetes_Habits.pdf as Changing_Diabetes_Habits.txt
Processed and saved: Diabetes.pdf as Diabetes.txt
Processed and saved: Glycemic_Index.pdf as Glycemic_Index.txt
Processed and saved: Prediabetes.pdf as Prediabetes.txt
Processed and saved: A1C_Test.pdf as A1C_Test.txt
Processed and saved: Diabetes_Healthy_Living.pdf as Diabetes_Healthy_Living.txt
Processed and saved: Prediabetes_additional_facts.pdf as Prediabetes_additional_facts.txt
Processed and saved: Medicines.pdf as Medicines.txt
Processed and saved: Diabetes_Management_Travelling.pdf as Diabetes_Management_Travelling.txt
Processed and saved: dgac381.pdf as dgac381.txt
Processed and saved: High_Sugar_Self_Care.pdf as High_Sugar_Self_Care.txt
Processed and saved: Diabetes_Risk_Factors.pdf as Diabetes_Risk_Factors.txt
Processed and saved: Diabetes_Type2_Meal_Planning.pdf as Diabetes_Type2_Meal_Planning.txt
Processed and saved: Diabetes_Nutrit

In [ ]:
#This part is to upload ALL the .txt files from the local machine to our drive
#Back transfer the text files to the Google Drive folder
#Grant the drive API edit acess and not read only
#Here, I advise you trash the token.json since we're using different SCOPES

SCOPES = ['https://www.googleapis.com/auth/drive']
#SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

def create_service():
    creds = None
    
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, I'm gonna log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('drive', 'v3', credentials=creds)
    return service

# save service instance
service_upload = create_service()



def upload_all_txt_files(service, folder_id):
    # Get the list of all files in the current working directory
    cwd = os.getcwd()
    files = [f for f in os.listdir(cwd) if os.path.isfile(os.path.join(cwd, f)) and f.endswith('.txt')]

    # List all files in the Google Drive folder to check for duplicates
    existing_files = service.files().list(q=f"'{folder_id}' in parents",
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name)').execute()
    existing_file_names = [file['name'] for file in existing_files.get('files', [])]

    for file_name in files:
        if file_name not in existing_file_names:
            file_path = os.path.join(cwd, file_name)
            file_metadata = {'name': file_name, 'parents': [folder_id]}
            media = MediaFileUpload(file_path, mimetype='text/plain')
            file = service.files().create(body=file_metadata,
                                          media_body=media,
                                          fields='id').execute()
            print(f"{file_name} uploaded successfully with File ID: {file.get('id')}")
        else:
            print(f"{file_name} already exists in the folder. Skipping upload.")


upload_all_txt_files(service_upload, '1n_9nLO15KnypYNEHbj9Z8USrBoQZVliQ')


In [12]:
#This part is to download ALL the .txt files (not PDFs) from the drive to local machine
#Again,the folder ID should be the same (our present wd)


# If modifying these SCOPES, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']


def create_service():
    creds = None
    # Recall that token.json stores our access and refresh tokens, and is created automatically when the authorization flow 
    # completes for the first time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, we can log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('drive', 'v3', credentials=creds)
    return service


# let's save service instance to interact with Google Drive
service = create_service()


def download_all_text_and_docx_files(service, folder_id):
    page_token = None
    while True:
        # Query to list the text and docx files in the specified folder
        results = service.files().list(
            q=f"'{folder_id}' in parents and (mimeType='text/plain' or mimeType='application/vnd.openxmlformats-officedocument.wordprocessingml.document' or mimeType='application/vnd.google-apps.document')",
            pageSize=10,  # Keep as 10 (for fast download)
            fields="nextPageToken, files(id, name, mimeType)",
            pageToken=page_token  # Use the nextPageToken from the previous request
        ).execute()

        items = results.get('files', [])
        if not items:
            print('No files found.')
            break  # Exit if no files are found

        for item in items:
            file_name = item['name']
            file_mime_type = item['mimeType']
            
            # Ensure the file extension is correct for Google Docs files
            if file_mime_type == 'application/vnd.google-apps.document':
                file_name += '.docx'
                request = service.files().export_media(fileId=item['id'], mimeType='application/vnd.openxmlformats-officedocument.wordprocessingml.document')
            else:
                request = service.files().get_media(fileId=item['id'])

            # Check if the file already exists in the current directory
            if not os.path.exists(file_name):
                print(f"Downloading {file_name}...")
                fh = io.BytesIO()
                downloader = MediaIoBaseDownload(fh, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()
                    print(f"Download {int(status.progress() * 100)}%.")

                # Let's save the file to disk only if it doesn't already exist
                with open(f'txt_docs/{file_name}', 'wb') as f:
                    fh.seek(0)
                    f.write(fh.read())
            else:
                print(f"{file_name} already exists. Skipping download.")

        page_token = results.get('nextPageToken', None)
        if page_token is None:
            break  


# Apply the function
download_all_text_and_docx_files(service, '1n_9nLO15KnypYNEHbj9Z8USrBoQZVliQ')



Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.


In [ ]:
#Now we can start working on the text data